## Experiment 1
<br>
-- Numbers data with eye-blink removed vs Raw numbers data


In [ ]:
import json
import utils
from data_handler import plot_images

import numpy as np

from collections import defaultdict

from IPython.display import clear_output

### Data Load

In [ ]:
data_path = '/home/donghyun/eye_writing_classification/v2_dataset/200_points_dataset/'

with open(data_path + 'eog_raw_numbers_200.json') as f:
  eog_raw_numbers = json.load(f)

with open(data_path + 'eog_eb_remove_numbers_200.json') as f:
  eog_eb_remove_numbers = json.load(f)

with open(data_path + 'reference_data_200.json') as f:
  reference_data = json.load(f)

### Experiment

In [ ]:
# 10 times experiments with raw numbers and eye-blink removed numbers data using HybridBaseModel

class Config:
    split_ratio = 0.3
    ref_key = 'numbers'
    batch_size = 18               # fix : Not must be equaled with number of test pairs 
    n_batch = 100
    lr = 0.001
    model_type = 'HybridBaseModel'
    epochs = 100

times = 10

raw_numbers_dict = defaultdict(list)
ebr_numbers_dict = defaultdict(list)
for t in range(times):
    print('times :', t+1)
    cfg = Config()
    
    _, ebr_train_acc, ebr_train_loss, ebr_test_acc = utils.experiment(cfg, eog_eb_remove_numbers, reference_data)
    _, raw_train_acc, raw_train_loss, raw_test_acc = utils.experiment(cfg, eog_raw_numbers, reference_data)

    ebr_numbers_dict[t] = [ebr_train_acc, ebr_train_loss, ebr_test_acc]
    raw_numbers_dict[t] = [raw_train_acc, raw_train_loss, raw_test_acc]


### Save

In [ ]:
import json

save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

with open(save_path + 'experiment1_raw_numbers_results.json', 'w') as f:
    json.dump(dict(raw_numbers_dict),f)

with open(save_path + 'experiment1_eb_removed_numbers_results.json', 'w') as f:
    json.dump(dict(ebr_numbers_dict),f)

### Visualization

In [ ]:
save_path = '/home/donghyun/eye_writing_classification/experiments/save/'

with open(save_path+'experiment1_raw_numbers_results.json') as f:
    raw_numbers_results = json.load(f)

with open(save_path+'experiment1_eb_removed_numbers_results.json') as f:
    ebr_numbers_results = json.load(f)


In [ ]:
raw_test_acc = []
ebr_test_acc = []
for t in range(10):
    key = str(t)
    raw_test_acc.append(raw_numbers_results[key][2])
    ebr_test_acc.append(ebr_numbers_results[key][2])

raw_avg_results = np.array(raw_test_acc).mean(axis=0)
ebr_avg_results  =np.array(ebr_test_acc).mean(axis=0)

In [ ]:
def analysis(data_list):
    return np.mean(data_list), max(data_list), min(data_list), np.std(data_list)

raw_numbers_test_performance = [t[-1] for t in raw_test_acc]
ebr_numbers_test_performance = [t[-1] for t in ebr_test_acc]

print('Accuracy with 10 repetitions')
print(' '*29 +'1,     2,    3,      4,      5,      6,     7,     8,     9,     10,       Avg.   Best.   Worst.  Std.')
print('without eye-blink removal : {}, {}'.format(raw_numbers_test_performance, analysis(raw_numbers_test_performance)))
print('with eye-blink removal    : {}, {}'.format(ebr_numbers_test_performance, analysis(ebr_numbers_test_performance)))

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1,2, figsize = (20,8))

# test accuracy
axes[0].plot(raw_avg_results, c = 'b', linestyle = 'solid', linewidth = 3)
axes[0].plot(ebr_avg_results, c = 'r', linestyle = 'solid', linewidth = 3)

axes[0].set_ylim(20,100)

axes[0].set_title("Evaluation", fontsize=20)
axes[0].set_xlabel('Epoch', fontsize = 20)
axes[0].set_ylabel('Accuracy', fontsize = 20)

axes[0].legend(['without eye-blink removal', 'with eye-blink removal'], fontsize = 15)

# plot
plt.show()